In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.stats import linregress
import scipy.stats as st

In [2]:
# Store filepath in a variable
data = "Resources/data.csv"
data_by_artist = "Resources/data_by_artist.csv"
data_by_genres = "Resources/data_by_genres.csv"
data_by_year = "Resources/data_by_year.csv"
data_w_genres = "Resources/data_w_genres.csv"

In [3]:
data_df = pd.read_csv(data, encoding="ISO-8859-1")
data_by_artist_df = pd.read_csv(data_by_artist, encoding="ISO-8859-1")
data_by_genres_df = pd.read_csv(data_by_genres, encoding="ISO-8859-1")
data_by_year_df = pd.read_csv(data_by_year, encoding="ISO-8859-1")
data_w_genres_df = pd.read_csv(data_w_genres, encoding="ISO-8859-1")

In [4]:
# view data by artist
data_df

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
0,0.991000,['Mamie Smith'],0.598,168333,0.224,0,0cS0A1fUEUd1EW3FcF8AEI,0.000522,5,0.3790,-12.628,0,Keep A Song In Your Soul,12,1920,0.0936,149.976,0.6340,1920
1,0.643000,"[""Screamin' Jay Hawkins""]",0.852,150200,0.517,0,0hbkKFIJm7Z05H8Zl9w30f,0.026400,5,0.0809,-7.261,0,I Put A Spell On You,7,1920-01-05,0.0534,86.889,0.9500,1920
2,0.993000,['Mamie Smith'],0.647,163827,0.186,0,11m7laMUgmOKqI3oYzuhne,0.000018,0,0.5190,-12.098,1,Golfing Papa,4,1920,0.1740,97.600,0.6890,1920
3,0.000173,['Oscar Velazquez'],0.730,422087,0.798,0,19Lc5SfJJ5O1oaxY0fpwfh,0.801000,2,0.1280,-7.311,1,True House Music - Xavier Santos & Carlos Gomi...,17,1920-01-01,0.0425,127.997,0.0422,1920
4,0.295000,['Mixe'],0.704,165224,0.707,1,2hJjbsLCytGsnAHfdsLejp,0.000246,10,0.4020,-6.036,0,Xuniverxe,2,1920-10-01,0.0768,122.076,0.2990,1920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174384,0.009170,"['DJ Combo', 'Sander-7', 'Tony T']",0.792,147615,0.866,0,46LhBf6TvYjZU2SMvGZAbn,0.000060,6,0.1780,-5.089,0,The One,0,2020-12-25,0.0356,125.972,0.1860,2020
174385,0.795000,['Alessia Cara'],0.429,144720,0.211,0,7tue2Wemjd0FZzRtDrQFZd,0.000000,4,0.1960,-11.665,1,A Little More,0,2021-01-22,0.0360,94.710,0.2280,2021
174386,0.806000,['Roger Fly'],0.671,218147,0.589,0,48Qj61hOdYmUCFJbpQ29Ob,0.920000,4,0.1130,-12.393,0,Together,0,2020-12-09,0.0282,108.058,0.7140,2020
174387,0.920000,['Taylor Swift'],0.462,244000,0.240,1,1gcyHQpBQ1lfXGdhZmWrHP,0.000000,0,0.1130,-12.077,1,champagne problems,69,2021-01-07,0.0377,171.319,0.3200,2021


In [5]:
def tidy_split(df, column, sep='|', keep=False):
    """
    Split the values of a column and expand so the new DataFrame has one split
    value per row. Filters rows where the column is missing.

    Params
    ------
    df : pandas.DataFrame
        dataframe with the column to split and expand
    column : str
        the column to split and expand
    sep : str
        the string used to split the column's values
    keep : bool
        whether to retain the presplit value as it's own row

    Returns
    -------
    pandas.DataFrame
        Returns a dataframe with the same columns as `df`.
    """
    indexes = list()
    new_values = list()
    df = df.dropna(subset=[column])
    for i, presplit in enumerate(df[column].astype(str)):
        values = presplit.split(sep)
        if keep and len(values) > 1:
            indexes.append(i)
            new_values.append(presplit)
        for value in values:
            indexes.append(i)
            new_values.append(value)
    new_df = df.iloc[indexes, :].copy()
    new_df[column] = new_values
    return new_df

In [6]:
data_df["artists"] = data_df["artists"].str.strip("[")
data_df["artists"] = data_df["artists"].str.strip("]")
data_df["artists"]

artists = tidy_split(data_df,"artists",sep=', ',keep=True)
artists["artists"] = artists["artists"].str.strip("'")
artists["artists"] = artists["artists"].str.strip('"')
artists

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
0,0.991000,Mamie Smith,0.598,168333,0.224,0,0cS0A1fUEUd1EW3FcF8AEI,0.000522,5,0.3790,-12.628,0,Keep A Song In Your Soul,12,1920,0.0936,149.976,0.6340,1920
1,0.643000,Screamin' Jay Hawkins,0.852,150200,0.517,0,0hbkKFIJm7Z05H8Zl9w30f,0.026400,5,0.0809,-7.261,0,I Put A Spell On You,7,1920-01-05,0.0534,86.889,0.9500,1920
2,0.993000,Mamie Smith,0.647,163827,0.186,0,11m7laMUgmOKqI3oYzuhne,0.000018,0,0.5190,-12.098,1,Golfing Papa,4,1920,0.1740,97.600,0.6890,1920
3,0.000173,Oscar Velazquez,0.730,422087,0.798,0,19Lc5SfJJ5O1oaxY0fpwfh,0.801000,2,0.1280,-7.311,1,True House Music - Xavier Santos & Carlos Gomi...,17,1920-01-01,0.0425,127.997,0.0422,1920
4,0.295000,Mixe,0.704,165224,0.707,1,2hJjbsLCytGsnAHfdsLejp,0.000246,10,0.4020,-6.036,0,Xuniverxe,2,1920-10-01,0.0768,122.076,0.2990,1920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174384,0.009170,Tony T,0.792,147615,0.866,0,46LhBf6TvYjZU2SMvGZAbn,0.000060,6,0.1780,-5.089,0,The One,0,2020-12-25,0.0356,125.972,0.1860,2020
174385,0.795000,Alessia Cara,0.429,144720,0.211,0,7tue2Wemjd0FZzRtDrQFZd,0.000000,4,0.1960,-11.665,1,A Little More,0,2021-01-22,0.0360,94.710,0.2280,2021
174386,0.806000,Roger Fly,0.671,218147,0.589,0,48Qj61hOdYmUCFJbpQ29Ob,0.920000,4,0.1130,-12.393,0,Together,0,2020-12-09,0.0282,108.058,0.7140,2020
174387,0.920000,Taylor Swift,0.462,244000,0.240,1,1gcyHQpBQ1lfXGdhZmWrHP,0.000000,0,0.1130,-12.077,1,champagne problems,69,2021-01-07,0.0377,171.319,0.3200,2021


In [7]:
# view data by genre
data_by_genres_df

,genres,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode
0,21st century classical,0.754600,0.284100,3.525932e+05,0.159580,0.484374,0.168580,-22.153400,0.062060,91.351000,0.143380,6.600000,4,1
1,432hz,0.485515,0.312000,1.047430e+06,0.391678,0.477250,0.265940,-18.131267,0.071717,118.900933,0.236483,41.200000,11,1
2,8-bit,0.028900,0.673000,1.334540e+05,0.950000,0.630000,0.069000,-7.899000,0.292000,192.816000,0.997000,0.000000,5,1
3,[],0.535793,0.546937,2.495312e+05,0.485430,0.278442,0.220970,-11.624754,0.101511,116.068980,0.486361,12.350770,7,1
4,a cappella,0.694276,0.516172,2.018391e+05,0.330533,0.036080,0.222983,-12.656547,0.083627,105.506031,0.454077,39.086248,7,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3227,zim urban groove,0.003910,0.553000,4.267200e+04,0.942000,0.961000,0.113000,-8.004000,0.039900,134.995000,0.752000,9.000000,7,1
3228,zolo,0.208648,0.533837,2.641016e+05,0.620470,0.163334,0.201430,-10.878906,0.061828,126.765194,0.576721,31.108254,9,1
3229,zouk,0.272928,0.641889,4.416418e+05,0.695778,0.257604,0.166011,-9.518889,0.050511,105.848889,0.878444,32.555556,7,1
3230,zurich indie,0.993000,0.705667,1.984173e+05,0.172667,0.468633,0.179667,-11.453333,0.348667,91.278000,0.739000,0.000000,7,0


In [8]:
# view data by years
data_by_year_df

,year,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode
0,1920,0.631242,0.515750,238092.997135,0.418700,0.354219,0.216049,-12.654020,0.082984,113.226900,0.498210,0.610315,2,1
1,1921,0.862105,0.432171,257891.762821,0.241136,0.337158,0.205219,-16.811660,0.078952,102.425397,0.378276,0.391026,2,1
2,1922,0.828934,0.575620,140135.140496,0.226173,0.254776,0.256662,-20.840083,0.464368,100.033149,0.571190,0.090909,5,1
3,1923,0.957247,0.577341,177942.362162,0.262406,0.371733,0.227462,-14.129211,0.093949,114.010730,0.625492,5.205405,0,1
4,1924,0.940200,0.549894,191046.707627,0.344347,0.581701,0.235219,-14.231343,0.092089,120.689572,0.663725,0.661017,10,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,2017,0.207413,0.580475,243402.249536,0.686392,0.247068,0.231513,-8.026319,0.096717,121.829807,0.436313,31.662338,1,1
98,2018,0.233836,0.602731,227073.140015,0.659552,0.229874,0.231009,-8.173434,0.121045,123.644288,0.426566,27.599484,7,1
99,2019,0.261344,0.603101,225463.326320,0.629781,0.221123,0.218501,-8.527674,0.106820,122.419702,0.457813,32.346930,7,1
100,2020,0.202303,0.605285,218977.482767,0.674247,0.249008,0.234415,-8.077563,0.100629,124.058308,0.468030,24.064741,0,1


In [9]:
#view data with genres
data_w_genres_df

,artists,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode,count,genres
0,"""Cats"" 1981 Original London Cast",0.598500,0.470100,267072.000000,0.376203,0.010261,0.283050,-14.434300,0.209150,114.128800,0.358320,38.200000,5,1,10,['show tunes']
1,"""Cats"" 1983 Broadway Cast",0.862538,0.441731,287280.000000,0.406808,0.081158,0.315215,-10.690000,0.176212,103.044154,0.268865,31.538462,5,1,26,[]
2,"""Fiddler On The Roofâ Motion Picture Chorus",0.856571,0.348286,328920.000000,0.286571,0.024593,0.325786,-15.230714,0.118514,77.375857,0.354857,34.571429,0,1,7,[]
3,"""Fiddler On The Roofâ Motion Picture Orchestra",0.884926,0.425074,262890.962963,0.245770,0.073587,0.275481,-15.639370,0.123200,88.667630,0.372030,34.407407,0,1,27,[]
4,"""Joseph And The Amazing Technicolor Dreamcoat""...",0.510714,0.467143,270436.142857,0.488286,0.009400,0.195000,-10.236714,0.098543,122.835857,0.482286,42.000000,5,1,7,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32534,ê¹í¨ê·¼,0.939000,0.174000,341821.000000,0.321000,0.756000,0.695000,-13.411000,0.042200,138.300000,0.055400,0.000000,3,1,1,[]
32535,ëì¨,0.717000,0.591500,241281.500000,0.473500,0.000000,0.105000,-6.588500,0.053700,107.599000,0.507500,16.000000,1,1,4,[]
32536,ë¯¸ì¤í°,0.416000,0.686000,216715.000000,0.822000,0.000000,0.110000,-6.300000,0.080200,122.047000,0.625000,0.000000,5,0,2,[]
32537,ìì¨ Zion & íí´ Hanhae of í¬í  Phantom,0.288000,0.739000,242480.000000,0.475000,0.000000,0.075700,-8.920000,0.379000,81.490000,0.800000,8.000000,2,0,2,[]


In [10]:
merge_df = pd.merge(artists, data_w_genres_df, on="artists", how="outer")
merge_df

,acousticness_x,artists,danceability_x,duration_ms_x,energy_x,explicit,id,instrumentalness_x,key_x,liveness_x,...,liveness_y,loudness_y,speechiness_y,tempo_y,valence_y,popularity_y,key_y,mode_y,count,genres
0,0.991,Mamie Smith,0.598,168333.0,0.2240,0.0,0cS0A1fUEUd1EW3FcF8AEI,0.000522,5.0,0.379,...,0.372625,-12.800313,0.121438,113.009938,0.641062,3.1875,0.0,1.0,16.0,"['harlem renaissance', 'traditional blues']"
1,0.993,Mamie Smith,0.647,163827.0,0.1860,0.0,11m7laMUgmOKqI3oYzuhne,0.000018,0.0,0.519,...,0.372625,-12.800313,0.121438,113.009938,0.641062,3.1875,0.0,1.0,16.0,"['harlem renaissance', 'traditional blues']"
2,0.992,Mamie Smith,0.782,195200.0,0.0573,0.0,5DlCyqLyX2AOVDTjjkDZ8x,0.000002,5.0,0.176,...,0.372625,-12.800313,0.121438,113.009938,0.641062,3.1875,0.0,1.0,16.0,"['harlem renaissance', 'traditional blues']"
3,0.995,Mamie Smith,0.482,198000.0,0.2290,0.0,0lqEx4vktZP1y9hnwfF27Y,0.000061,7.0,0.549,...,0.372625,-12.800313,0.121438,113.009938,0.641062,3.1875,0.0,1.0,16.0,"['harlem renaissance', 'traditional blues']"
4,0.992,Mamie Smith,0.574,189800.0,0.1380,0.0,4HYmmG8uHL2hP4zSFWavKF,0.000492,3.0,0.233,...,0.372625,-12.800313,0.121438,113.009938,0.641062,3.1875,0.0,1.0,16.0,"['harlem renaissance', 'traditional blues']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273323,NaN,"Victor Yturbe ""El Piruli""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.176600,-12.227800,0.029280,98.380400,0.509000,34.2000,9.0,1.0,10.0,"['bolero', 'bolero mexicano', 'ranchera']"
273324,NaN,"Weiss, George D.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.184000,-9.793000,0.032600,171.248000,0.242000,0.0000,3.0,1.0,1.0,[]
273325,NaN,"Yessister, Jazzsister",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.082567,-9.682333,0.032133,147.673000,0.249000,0.0000,5.0,0.0,3.0,[]
273326,NaN,\,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.332000,-15.643000,0.048800,113.235000,0.487000,0.0000,4.0,1.0,1.0,[]


In [11]:
merge_df.columns

Index(['acousticness_x', 'artists', 'danceability_x', 'duration_ms_x',
       'energy_x', 'explicit', 'id', 'instrumentalness_x', 'key_x',
       'liveness_x', 'loudness_x', 'mode_x', 'name', 'popularity_x',
       'release_date', 'speechiness_x', 'tempo_x', 'valence_x', 'year',
       'acousticness_y', 'danceability_y', 'duration_ms_y', 'energy_y',
       'instrumentalness_y', 'liveness_y', 'loudness_y', 'speechiness_y',
       'tempo_y', 'valence_y', 'popularity_y', 'key_y', 'mode_y', 'count',
       'genres'],
      dtype='object')

In [12]:
reordered_merge_df = merge_df[["artists", "year", "release_date", "genres", "name", "id", "popularity_x", "popularity_y", "danceability_x", "danceability_y", "tempo_x", "tempo_y", "energy_x", "energy_y", "acousticness_x", "acousticness_y", "duration_ms_x", "duration_ms_y", "explicit", "instrumentalness_x", "instrumentalness_y", "key_x", "key_y", "liveness_x", "liveness_y", "loudness_x", "loudness_y", "speechiness_x", "speechiness_y", "valence_x", "valence_y"]]


In [13]:
renamed_merge_df = reordered_merge_df.rename(columns={"popularity_x": "Popularity of Song",
                                        "popularity_y": "Avg Popularity of Artist",
                                        "danceability_x": "Danceability of Song",
                                        "danceability_y": "Avg Danceability of Artist",
                                        "tempo_x": "Tempo of Song",
                                        "tempo_y": "Avg Tempo of Artist",
                                        "energy_x": "Energy of Song",
                                        "energy_y": "Avg Energy of Artist",
                                        "acousticness_x": "Acousticness of Song", 
                                        "acousticness_y": "Avg Acousticness of Artist",
                                        "duration_ms_x": "Duration of Song",
                                        "duration_ms_y": "Avg Duration of Artist",
                                        "danceability_x": "Danceability of Song",
                                        "danceability_y": "Avg Danceability of Artist",
                                        "tempo_x": "Tempo of Song",
                                        "tempo_y": "Avg Tempo of Artist",
                                        "instrumentalness_x": "Three Star Reviews",
                                        "instrumentalness_y": "Four Star Reviews",
                                        "key_x": "Key of Song",
                                        "key_y": "Avg Key of Artist",
                                        "loudness_x": "Loudness of Song",
                                        "loudness_y": "Avg Loudness of Artist",
                                        "speechiness_x": "Speechiness of Song",
                                        "speechiness_y": "Avg Speechiness of Artist",
                                        "valence_x": "Valence of Song",
                                        "valence_y": "Avg Valence of Artist",
                                        "liveness_x": "Liveness of Song",
                                        "liveness_y": "Avg Liveness of Artist",
                                       })

In [14]:
renamed_merge_df

,artists,year,release_date,genres,name,id,Popularity of Song,Avg Popularity of Artist,Danceability of Song,Avg Danceability of Artist,...,Key of Song,Avg Key of Artist,Liveness of Song,Avg Liveness of Artist,Loudness of Song,Avg Loudness of Artist,Speechiness of Song,Avg Speechiness of Artist,Valence of Song,Avg Valence of Artist
0,Mamie Smith,1920.0,1920,"['harlem renaissance', 'traditional blues']",Keep A Song In Your Soul,0cS0A1fUEUd1EW3FcF8AEI,12.0,3.1875,0.598,0.627250,...,5.0,0.0,0.379,0.372625,-12.628,-12.800313,0.0936,0.121438,0.634,0.641062
1,Mamie Smith,1920.0,1920,"['harlem renaissance', 'traditional blues']",Golfing Papa,11m7laMUgmOKqI3oYzuhne,4.0,3.1875,0.647,0.627250,...,0.0,0.0,0.519,0.372625,-12.098,-12.800313,0.1740,0.121438,0.689,0.641062
2,Mamie Smith,1920.0,1920,"['harlem renaissance', 'traditional blues']",Don't You Advertise Your Man,5DlCyqLyX2AOVDTjjkDZ8x,5.0,3.1875,0.782,0.627250,...,5.0,0.0,0.176,0.372625,-12.453,-12.800313,0.0592,0.121438,0.487,0.641062
3,Mamie Smith,1920.0,1920,"['harlem renaissance', 'traditional blues']",Kansas City Man Blues - 78rpm Version,0lqEx4vktZP1y9hnwfF27Y,4.0,3.1875,0.482,0.627250,...,7.0,0.0,0.549,0.372625,-12.619,-12.800313,0.0812,0.121438,0.461,0.641062
4,Mamie Smith,1920.0,1920,"['harlem renaissance', 'traditional blues']",Miss Jenny's Ball (aka There'll Be No Freebies...,4HYmmG8uHL2hP4zSFWavKF,1.0,3.1875,0.574,0.627250,...,3.0,0.0,0.233,0.372625,-14.171,-12.800313,0.1090,0.121438,0.764,0.641062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273323,"Victor Yturbe ""El Piruli""",NaN,NaN,"['bolero', 'bolero mexicano', 'ranchera']",NaN,NaN,NaN,34.2000,NaN,0.521000,...,NaN,9.0,NaN,0.176600,NaN,-12.227800,NaN,0.029280,NaN,0.509000
273324,"Weiss, George D.",NaN,NaN,[],NaN,NaN,NaN,0.0000,NaN,0.296000,...,NaN,3.0,NaN,0.184000,NaN,-9.793000,NaN,0.032600,NaN,0.242000
273325,"Yessister, Jazzsister",NaN,NaN,[],NaN,NaN,NaN,0.0000,NaN,0.256333,...,NaN,5.0,NaN,0.082567,NaN,-9.682333,NaN,0.032133,NaN,0.249000
273326,\,NaN,NaN,[],NaN,NaN,NaN,0.0000,NaN,0.335000,...,NaN,4.0,NaN,0.332000,NaN,-15.643000,NaN,0.048800,NaN,0.487000


In [15]:
renamed_merge_df["genres"].replace('\'','', regex=True, inplace=True)
renamed_merge_df["genres"] = renamed_merge_df["genres"].str.strip("[")
renamed_merge_df["genres"] = renamed_merge_df["genres"].str.strip("]")
renamed_merge_df

,artists,year,release_date,genres,name,id,Popularity of Song,Avg Popularity of Artist,Danceability of Song,Avg Danceability of Artist,...,Key of Song,Avg Key of Artist,Liveness of Song,Avg Liveness of Artist,Loudness of Song,Avg Loudness of Artist,Speechiness of Song,Avg Speechiness of Artist,Valence of Song,Avg Valence of Artist
0,Mamie Smith,1920.0,1920,"harlem renaissance, traditional blues",Keep A Song In Your Soul,0cS0A1fUEUd1EW3FcF8AEI,12.0,3.1875,0.598,0.627250,...,5.0,0.0,0.379,0.372625,-12.628,-12.800313,0.0936,0.121438,0.634,0.641062
1,Mamie Smith,1920.0,1920,"harlem renaissance, traditional blues",Golfing Papa,11m7laMUgmOKqI3oYzuhne,4.0,3.1875,0.647,0.627250,...,0.0,0.0,0.519,0.372625,-12.098,-12.800313,0.1740,0.121438,0.689,0.641062
2,Mamie Smith,1920.0,1920,"harlem renaissance, traditional blues",Don't You Advertise Your Man,5DlCyqLyX2AOVDTjjkDZ8x,5.0,3.1875,0.782,0.627250,...,5.0,0.0,0.176,0.372625,-12.453,-12.800313,0.0592,0.121438,0.487,0.641062
3,Mamie Smith,1920.0,1920,"harlem renaissance, traditional blues",Kansas City Man Blues - 78rpm Version,0lqEx4vktZP1y9hnwfF27Y,4.0,3.1875,0.482,0.627250,...,7.0,0.0,0.549,0.372625,-12.619,-12.800313,0.0812,0.121438,0.461,0.641062
4,Mamie Smith,1920.0,1920,"harlem renaissance, traditional blues",Miss Jenny's Ball (aka There'll Be No Freebies...,4HYmmG8uHL2hP4zSFWavKF,1.0,3.1875,0.574,0.627250,...,3.0,0.0,0.233,0.372625,-14.171,-12.800313,0.1090,0.121438,0.764,0.641062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273323,"Victor Yturbe ""El Piruli""",NaN,NaN,"bolero, bolero mexicano, ranchera",NaN,NaN,NaN,34.2000,NaN,0.521000,...,NaN,9.0,NaN,0.176600,NaN,-12.227800,NaN,0.029280,NaN,0.509000
273324,"Weiss, George D.",NaN,NaN,,NaN,NaN,NaN,0.0000,NaN,0.296000,...,NaN,3.0,NaN,0.184000,NaN,-9.793000,NaN,0.032600,NaN,0.242000
273325,"Yessister, Jazzsister",NaN,NaN,,NaN,NaN,NaN,0.0000,NaN,0.256333,...,NaN,5.0,NaN,0.082567,NaN,-9.682333,NaN,0.032133,NaN,0.249000
273326,\,NaN,NaN,,NaN,NaN,NaN,0.0000,NaN,0.335000,...,NaN,4.0,NaN,0.332000,NaN,-15.643000,NaN,0.048800,NaN,0.487000


In [16]:
new_artists = tidy_split(renamed_merge_df,"genres",sep=', ',keep=False)
renamed_merge_df["genres"] = renamed_merge_df["genres"].str.lstrip()
new_artists

,artists,year,release_date,genres,name,id,Popularity of Song,Avg Popularity of Artist,Danceability of Song,Avg Danceability of Artist,...,Key of Song,Avg Key of Artist,Liveness of Song,Avg Liveness of Artist,Loudness of Song,Avg Loudness of Artist,Speechiness of Song,Avg Speechiness of Artist,Valence of Song,Avg Valence of Artist
0,Mamie Smith,1920.0,1920,harlem renaissance,Keep A Song In Your Soul,0cS0A1fUEUd1EW3FcF8AEI,12.0,3.1875,0.598,0.627250,...,5.0,0.0,0.379,0.372625,-12.628,-12.800313,0.0936,0.121438,0.634,0.641062
0,Mamie Smith,1920.0,1920,traditional blues,Keep A Song In Your Soul,0cS0A1fUEUd1EW3FcF8AEI,12.0,3.1875,0.598,0.627250,...,5.0,0.0,0.379,0.372625,-12.628,-12.800313,0.0936,0.121438,0.634,0.641062
1,Mamie Smith,1920.0,1920,harlem renaissance,Golfing Papa,11m7laMUgmOKqI3oYzuhne,4.0,3.1875,0.647,0.627250,...,0.0,0.0,0.519,0.372625,-12.098,-12.800313,0.1740,0.121438,0.689,0.641062
1,Mamie Smith,1920.0,1920,traditional blues,Golfing Papa,11m7laMUgmOKqI3oYzuhne,4.0,3.1875,0.647,0.627250,...,0.0,0.0,0.519,0.372625,-12.098,-12.800313,0.1740,0.121438,0.689,0.641062
2,Mamie Smith,1920.0,1920,harlem renaissance,Don't You Advertise Your Man,5DlCyqLyX2AOVDTjjkDZ8x,5.0,3.1875,0.782,0.627250,...,5.0,0.0,0.176,0.372625,-12.453,-12.800313,0.0592,0.121438,0.487,0.641062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273323,"Victor Yturbe ""El Piruli""",NaN,NaN,ranchera,NaN,NaN,NaN,34.2000,NaN,0.521000,...,NaN,9.0,NaN,0.176600,NaN,-12.227800,NaN,0.029280,NaN,0.509000
273324,"Weiss, George D.",NaN,NaN,,NaN,NaN,NaN,0.0000,NaN,0.296000,...,NaN,3.0,NaN,0.184000,NaN,-9.793000,NaN,0.032600,NaN,0.242000
273325,"Yessister, Jazzsister",NaN,NaN,,NaN,NaN,NaN,0.0000,NaN,0.256333,...,NaN,5.0,NaN,0.082567,NaN,-9.682333,NaN,0.032133,NaN,0.249000
273326,\,NaN,NaN,,NaN,NaN,NaN,0.0000,NaN,0.335000,...,NaN,4.0,NaN,0.332000,NaN,-15.643000,NaN,0.048800,NaN,0.487000


In [19]:
new_artists.to_csv(path_or_buf="Resources/artist_genres.csv", index=False)